# แบบฝึกหัด 3

---

In [130]:
import numpy as np
import pandas as pd
import statistics

In [131]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

###  

3.1 บริษัทให้บริการด้าน IT ครบวงจรแห่งหนึ่งได้เก็บรวบรวมข้อมูลค่าซ่อมบำรุงอุปกรณ์คอมพิวเตอร์แม่ข่ายและจำนวนลูกค้าของบริษัทในปีที่ผ่านมาเป็นดังไฟล์ ex3_1_itcost.csv  
จงแสดงวิธีหาค่ามัธยฐานและฐานนิยมของค่าซ่อมบำรุงอุปกรณ์คอมพิวเตอร์แม่ข่าย

In [132]:
df_it = pd.read_csv('ex3_1_itcost.csv')

df_it

,maintenance_cost,num_customer
0,<=5000,33
1,5001-10000,41
2,10001-15000,20
3,15001-20000,17
4,20001-25000,10
5,25001-30000,16
6,>=30001,5


In [133]:
#split "-", "<=", ">="
df_it['maintenance_cost'] = df_it['maintenance_cost'].str.replace('<=', '-')
df_it['maintenance_cost'] = df_it['maintenance_cost'].str.replace('>=', '')
df_it[['cost_from','cost_to']] = df_it['maintenance_cost'].str.split('-',expand=True)
df_it['num_customer_cum'] = df_it['num_customer'].cumsum()
#display(df_it)
# หาค่ามัธยฐาน 
med_pos = df_it.num_customer.sum()/2
for i in df_it.index:
    if df_it.num_customer_cum[i] >= med_pos:
        e=i
        break
lower = (int(df_it.cost_from[e])+int(df_it.cost_to[e-1]))/2
lower = lower+( (med_pos - df_it.num_customer_cum[e-1]) / df_it.num_customer[e])*5000
print("ค่ามัธยฐาน: {:.2f}".format(lower))

# ฐานนิยม
e = np.argmax(df_it.num_customer)
lower = (int(df_it.cost_from[e])+int(df_it.cost_to[e-1]))/2
lower += ( df_it.num_customer[e-1]/(df_it.num_customer[e-1]+df_it.num_customer[e+1]) )*5000
print("ค่าฐานนิยม: {:.2f}".format(lower))

ค่ามัธยฐาน: 9634.65
ค่าฐานนิยม: 8113.71


###   

3.2 สุ่มตัวอย่างนักศึกษามาจำนวนหนึ่ง พบว่าคะแนนสอบวิชาสถิติ เป็นดังตาราง 'ex3_2_score.csv' จงหาค่า

**3.2.1 ค่าเฉลี่ย**

In [136]:
df_score = pd.read_csv('ex3_2_score.csv')
boundary = df_score.score.str.split('-', expand=True).astype(int)
df_score['ค่ากึ่งกลาง (x)'] = (boundary[0] + boundary[1]) / 2
df_score['x^2'] = df_score['ค่ากึ่งกลาง (x)'] ** 2
df_score['fx'] = df_score['ค่ากึ่งกลาง (x)'] * df_score['num_student']
df_score['fx^2'] = df_score['fx'] ** 2
df_score['num_student_cum'] = df_score['num_student'].cumsum()
df_score[['score_from', 'score_to']] = df_score['score'].str.split('-', expand=True)

In [137]:
mean = df_score['fx'].sum() / df_score['num_student'].sum()
mean

25.85

**3.2.2 ค่ามัธยฐาน**

In [138]:
med_pos = df_score.num_student.sum()/2
for i in df_score.index:
    if df_score.num_student_cum[i] >= med_pos:
        e=i
        break
lower = (int(df_score.score_from[e])+int(df_score.score_to[e-1]))/2
lower = lower+( (med_pos - df_score.num_student_cum[e-1]) / df_score.num_student[e])*5
print("{:.2f}".format(lower))

25.78


**3.2.3 ค่าฐานนิยม**

In [139]:
e = np.argmax(df_score.num_student)
lower = (int(df_score.score_from[e])+int(df_score.score_to[e-1]))/2
lower + ( df_score.num_student[e-1]/(df_score.num_student[e-1]+df_score.num_student[e+1]) )*5

22.6875

**3.2.4 Q<sub>1</sub> และ Q<sub>3</sub>**

In [140]:
n = df_score.num_student.sum()
def calcul(pos, n, formula):
    q1_pos = n*eval(formula)
    lower = (int(df_score.score_to[pos-1])+int(df_score.score_from[pos])) / 2
    return lower+( (q1_pos-df_score.num_student_cum[pos-1])/df_score.num_student[pos] )*5

In [141]:

Q1 = calcul(1, 100, '1/4')
Q3 = calcul(4, 100, '3/4')
print("Q1 = {:.2f}".format(Q1))
print("Q3 = {:.2f}".format(Q3))



Q1 = 19.07
Q3 = 33.00


**3.2.5 D<sub>7</sub>**

In [142]:
print("D7 = {:.2f}".format(calcul(4, 100, '7/10')))

D7 = 31.44


**3.2.6 P<sub>75</sub>**

In [143]:
print("P75 = {:.2f}".format(calcul(4, 100, '75/100')))

P75 = 33.00


**3.2.7 พิสัย**

In [144]:
diff = 0.5
range_score = (df_score.score_to.astype(float).max() + diff) - (df_score.score_from.astype(float).min() - diff)
range_score

30.0

**3.2.8 ส่วนเบี่ยงเบนควอร์ไทล์**

In [145]:
print("{:.2f}".format((Q3 - Q1) / 2))

6.96


**3.2.9 ส่วนเบี่ยงเบนมาตราฐานและความแปรปรวน**

In [146]:
sigma_fx2 = sum( ((df_score.score_from.astype(int)+df_score.score_to.astype(int))/2)**2  * df_score.num_student)
s = np.sqrt((sigma_fx2 - n*mean**2) / (n-1))
print("ค่าเบี่ยงเบนมาตราฐาน: {:.2f}".format(s))
print("ค่าความแปรปรวน: {:.2f}".format(s**2))

ค่าเบี่ยงเบนมาตราฐาน: 8.36
ค่าความแปรปรวน: 69.83


**3.2.10 สัมประสิทธิ์ของการแปรผัน**

In [147]:
cv = s / (df_score.fx.sum()/100)
print("{:.2f}".format(cv))

0.32


###   

3.3 บริษัทเกมแห่งหนึ่งได้ทำการทดสอบ game engine รุ่นใหม่ ในคอมพิวเตอร์ที่มีส่วนประกอบของฮารด์แวร์ต่างๆกัน (เช่น cpu, gpu, ram, ฯลฯ) จำนวน 30 เครื่อง และทำการวัดค่า frames per second (FPS) ได้ผลดังนี้  
60.0, 30.2, 33.4, 59.9, 40.1, 25.5, 17.4, 28.6, 45.5, 57.8,  
22.5, 50.2, 30.8, 43.1, 50.5, 35.5, 27.4, 29.2, 55.5, 10.2,   
32.2, 27.8, 33.8, 53.4, 25.5, 30.4, 36.1, 49.3, 18.6, 50.0   
จงคำนวณหาค่าเฉลี่ย, ส่วนเบี่ยงเบนมาตราฐาน, ฐานนิยม, Q<sub>1</sub>, D<sub>3</sub>, P<sub>45</sub> ของค่า FPS ที่ได้จากการทดสอบ

ค่าเฉลี่ย:

In [148]:
fps = [60.0, 30.2, 33.4, 59.9, 40.1, 25.5, 17.4, 28.6, 45.5, 57.8,
22.5, 50.2, 30.8, 43.1, 50.5, 35.5, 27.4, 29.2, 55.5, 10.2,
32.2, 27.8, 33.8, 53.4, 25.5, 30.4, 36.1, 49.3, 18.6, 50.0]
statistics.mean(fps)

37.013333333333335

ส่วนเบี่ยงเบนมาตราฐาน:

In [149]:
np.std(fps, ddof=1)

13.51187812906528

ฐานนิยม:

In [150]:
statistics.mode(fps)

25.5

Q<sub>1</sub>, D<sub>3</sub> และ P<sub>45</sub>:  

In [151]:
print("{:.2f}".format(np.quantile(fps, 0.25)))
print("{:.2f}".format(np.percentile(fps, 30)))
print("{:.2f}".format(np.percentile(fps, 45)))


28.00
29.02
32.26


###   

3.4 หน่วยงานวิจัยแห่งหนึ่งได้รับมอบหมายจากทางรัฐบาลให้ทำการเปรียบเทียบระบบที่ใช้ในการผลิตพลังงานกระแสไฟฟ้าสำหรับนิคมอุตสาหกรรมแห่งหนึ่ง ระบบแรกมีกำลังการผลิตสูงสุด 10,000 เมกะวัตต์ โดยมีค่าเฉลี่ยและส่วนเบี่ยงเบนมาตราฐานของกำลังการผลิตอยู่ที่ 9,000±320 เมกะวัตต์ ระบบที่สองมีกำลังการผลิตสูงสุด 15,000 เมกะวัตต์ โดยมีค่าเฉลี่ยและส่วนเบี่ยงเบนมาตราฐานของกำลังการผลิตอยู่ที่ 14,000±900 เมกะวัตต์ จากข้อมูลนี้ทางสถาบันวิจัยจะสามารถสรุปว่าระบบใดมีกำลังการผลิตที่เที่ยงตรงกว่า



In [152]:
cv1 = 320 * 100 / 9000
cv2 = 900 * 100 / 14000
print("ระบบที่ 1 มีกำลังผลิตที่เที่ยงตรงกว่า" if cv1 < cv2 else "ระบบที่ 2 มีกำลังผลิตที่เที่ยงตรงกว่า")

ระบบที่ 1 มีกำลังผลิตที่เที่ยงตรงกว่า
